In [3]:
#!/usr/bin/env python3
"""
Correlation test between ViT and Mouse Brain PCA components.

 - Tests H0: corr(ViT PC_i, Brain PC_j) = 0
 - Uses all PCs explaining ≥90% variance in each space
 - Applies Benjamini–Hochberg correction for multiple testing
 - Prints significant pairs and summary statistics
"""

import os
import pickle
import numpy as np
from sklearn.decomposition import PCA
from scipy.special import softmax
from skbio.stats.composition import clr
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests

# ---------------------------------------------------------------
# CONFIG
# ---------------------------------------------------------------
VIT_PATH = '/home/maria/Documents/HuggingMouseData/MouseViTEmbeddings/google_vit-base-patch16-224_embeddings_logits.pkl'
NEURAL_PATH = '/home/maria/LuckyMouse/pixel_transformer_neuro/data/processed/hybrid_neural_responses.npy'
AREAS_PATH = '/home/maria/MITNeuralComputation/visualization/brain_area.npy'       # brain area label per neuron
AREA_NAME = 'VISam'                 # choose brain area
N_IMAGES, N_TRIALS = 118, 50
ALPHA_FDR = 0.05

# ---------------------------------------------------------------
# LOAD ViT EMBEDDINGS AND RUN PCA
# ---------------------------------------------------------------
print("🔹 Loading ViT embeddings...")
with open(VIT_PATH, 'rb') as f:
    vit_logits = pickle.load(f)['natural_scenes']
embeddings = np.asarray(vit_logits)
X = softmax(embeddings, axis=1)
X_clr = clr(X + 1e-12)

pca_vit_full = PCA(n_components=min(X_clr.shape), random_state=0)
pca_vit_full.fit(X_clr)
vit_cumvar = np.cumsum(pca_vit_full.explained_variance_ratio_)
vit_ncomp = np.searchsorted(vit_cumvar, 0.90) + 1
print(f"ViT PCs covering 90% variance: {vit_ncomp}")

vit_pca = PCA(n_components=vit_ncomp, random_state=0)
vit_U = vit_pca.fit_transform(X_clr)      # (images × vit_ncomp)
vit_var = vit_pca.explained_variance_ratio_

# ---------------------------------------------------------------
# LOAD AND FILTER BRAIN DATA
# ---------------------------------------------------------------
print(f"🔹 Loading neural responses ({AREA_NAME})...")
dat = np.load(NEURAL_PATH, mmap_mode='r')
areas = np.load(AREAS_PATH, allow_pickle=True)
mask = areas == AREA_NAME
dat = dat[mask]
n_neurons, n_total = dat.shape
n_time = n_total // (N_IMAGES * N_TRIALS)
dat = dat.reshape(n_neurons, N_IMAGES, N_TRIALS, n_time)
X_mean = dat.mean(axis=(2, 3))  # (neurons × images)

# ---------------------------------------------------------------
# PCA ON BRAIN DATA
# ---------------------------------------------------------------
pca_brain_full = PCA(n_components=min(X_mean.shape), random_state=0)
pca_brain_full.fit(X_mean - X_mean.mean(axis=0))
brain_cumvar = np.cumsum(pca_brain_full.explained_variance_ratio_)
brain_ncomp = np.searchsorted(brain_cumvar, 0.90) + 1
print(f"{AREA_NAME} PCs covering 90% variance: {brain_ncomp}")

pca_brain = PCA(n_components=brain_ncomp, random_state=0)
brain_V = pca_brain.fit_transform(X_mean - X_mean.mean(axis=0))  # (neurons × brain_ncomp)
brain_loadings = pca_brain.components_.T                         # (images × brain_ncomp)

# ---------------------------------------------------------------
# CORRELATION TESTS
# ---------------------------------------------------------------
print("🔹 Computing all pairwise correlations...")
r_vals, p_vals, pairs = [], [], []

for i in range(vit_ncomp):
    for j in range(brain_ncomp):
        r, p = pearsonr(vit_U[:, i], brain_loadings[:, j])
        r_vals.append(r)
        p_vals.append(p)
        pairs.append((i+1, j+1))  # 1-indexed for reporting

r_vals = np.array(r_vals)
p_vals = np.array(p_vals)

n_tests = len(p_vals)
print(f"Total tests performed: {n_tests}")

# ---------------------------------------------------------------
# MULTIPLE TESTING CORRECTION
# ---------------------------------------------------------------
reject, pvals_corrected, _, _ = multipletests(p_vals, alpha=ALPHA_FDR, method='fdr_bh')

sig_idx = np.where(reject)[0]
if len(sig_idx) == 0:
    print("❌ No correlations survived FDR correction.")
else:
    print(f"✅ {len(sig_idx)} / {n_tests} correlations survived FDR correction (q < {ALPHA_FDR}).")
    print("\nSignificant pairs (ViT PC ↔ Brain PC):")
    for idx in sig_idx:
        i, j = pairs[idx]
        print(f"  ViT PC{i:2d} ↔ {AREA_NAME} PC{j:2d} | r = {r_vals[idx]:+.3f}, p = {p_vals[idx]:.2e}, q = {pvals_corrected[idx]:.2e}")

# ---------------------------------------------------------------
# SAVE RESULTS
# ---------------------------------------------------------------
out = {
    "pairs": pairs,
    "r": r_vals,
    "p_raw": p_vals,
    "p_fdr": pvals_corrected,
    "significant": reject,
    "vit_ncomp": vit_ncomp,
    "brain_ncomp": brain_ncomp,
}
np.savez(f"vit_{AREA_NAME}_corr_results.npz", **out)
print(f"\n💾 Results saved to vit_{AREA_NAME}_corr_results.npz")


🔹 Loading ViT embeddings...
ViT PCs covering 90% variance: 44
🔹 Loading neural responses (VISam)...
VISam PCs covering 90% variance: 61
🔹 Computing all pairwise correlations...
Total tests performed: 2684
✅ 1 / 2684 correlations survived FDR correction (q < 0.05).

Significant pairs (ViT PC ↔ Brain PC):
  ViT PC 1 ↔ VISam PC 2 | r = +0.393, p = 1.08e-05, q = 2.90e-02

💾 Results saved to vit_VISam_corr_results.npz
